In [ ]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from iqx import *


# Simulation time!
backend = Aer.get_backend('qasm_simulator')
results = execute(grover_circuit, backend=backend, shots=1024).result()
answer = results.get_counts()
plot_histogram(answer)

backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= 3 and 
                                   not x.configuration().simulator and x.status().operational==True))
print("least busy backend: ", backend)

# Run our circuit on the least busy backend. Monitor the execution of the job in the queue
from qiskit.tools.monitor import job_monitor
job = execute(grover_circuit, backend=backend, shots=1024, optimization_level=3)
job_monitor(job, interval = 2)

# Get the results from the computation
results = job.result()
answer = results.get_counts(grover_circuit)
plot_histogram(answer)"""

In [ ]:
# initialization
# matplotlib is a Python module deisgned for plotting
# we are only using the pyplot submodule
# the 'as plt' just saves us from typing 'matplotlib.pyplot' over and over again
# similarly, the 'np' just saves us from typing 'numpy' over and over again
# numpy is a Python module designed for mathematical calculations (such as linear algebra)
import matplotlib.pyplot as plt 
import numpy as np

# importing Qiskit
# Qiskit is a special quantum computing programming language developed by IBM
# for running code intended for quantum computers using IBMQ
from qiskit import IBMQ, Aer, QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.providers.ibmq import least_busy

# import basic plot tools
from qiskit.visualization import plot_histogram

# Function to Initialize Quantum States for N Qubits
# Two parameters: QC, a quantum circuit, and Qubits, the number of qubits
def Initialize(QC, Qubits):
    # Apply a Hadamard (H) Gate to the Qubits
    # Hadamard gate just puts the qubit in a superposition
    for q in Qubits: # iterates through all the qubits,
        QC.h(q) # applying a H gate to each individually
    return QC # returns the new quantum circuit with the H gates applied

#Three Qubits
qc = QuantumCircuit(3) 
qc.cz(0, 2) # CZ on q0 and q2
qc.cz(1, 2) # cz on q1 and q2
oracle_ex3 = qc.to_gate() # return the oracle as a gate
oracle_ex3.name = "U$_\omega$"

def diffuser(nqubits): # nqubits is the number of qubits
    qc = QuantumCircuit(nqubits) # create a quantum circuit with 'nqubits' qubits
    # H gates
    for qubit in range(nqubits):
        qc.h(qubit)
    # X gates
    for qubit in range(nqubits):
        qc.x(qubit)
    # MCZ gate
    qc.h(nqubits-1)
    qc.mct(list(range(nqubits-1)), nqubits-1)  # this is a MCT gate
    qc.h(nqubits-1)
    # X gates again
    for qubit in range(nqubits):
        qc.x(qubit)
    # H gates again
    for qubit in range(nqubits):
        qc.h(qubit)
    # return the diffuser as a gate for convenience
    U_s = qc.to_gate()
    U_s.name = "U$_s$"
    return U_s

n = 3
grover_circuit = QuantumCircuit(n)
grover_circuit = Initialize(grover_circuit, [0,1,2])
grover_circuit.append(oracle_ex3, [0,1,2])
grover_circuit.append(diffuser(n), [0,1,2])
grover_circuit.measure_all()
grover_circuit.draw()


# run on real quantum computer
backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= 3 and 
                                   not x.configuration().simulator and x.status().operational==True))
print("least busy backend: ", backend)

# Run our circuit on the least busy backend. Monitor the execution of the job in the queue
from qiskit.tools.monitor import job_monitor
job = execute(grover_circuit, backend=backend, shots=1024, optimization_level=3)
job_monitor(job, interval = 2)

# Get the results from the computation
results = job.result()
answer = results.get_counts(grover_circuit)
plot_histogram(answer)